In [ ]:
!pip install sounddevice vosk faster-whisper soundfile
!apt-get install -y portaudio19-dev

In [ ]:
from gtts import gTTS
from IPython.display import Audio

tts = gTTS("Hello, this is a test audio for speech to text models.", lang="en")
tts.save("test_audio.mp3")

Audio("test_audio.mp3")


In [ ]:
!ffmpeg -i test_audio.mp3 -ar 16000 -ac 1 test.wav -y

In [ ]:
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip -q vosk-model-small-en-us-0.15.zip

from vosk import Model, KaldiRecognizer
import wave, json

wf = wave.open("test.wav", "rb")
rec = KaldiRecognizer(Model("vosk-model-small-en-us-0.15"), 16000)

result = ""
while True:
    data = wf.readframes(4000)
    if not data:
        break
    if rec.AcceptWaveform(data):
        result += json.loads(rec.Result())["text"] + " "

result += json.loads(rec.FinalResult())["text"]
print("VOSK:", result)


In [ ]:
!pip install -q faster-whisper

from faster_whisper import WhisperModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = WhisperModel("small", device=device)

segments, info = model.transcribe("test.wav")
text = " ".join([s.text for s in segments])

print("WHISPER:", text)
